In [3]:
import pandas as pd
import numpy as np
import xlwings as xw
import glob
import os
import tsfel
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import chi2
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC



### Data with signal

In [50]:
xl = r'C:\Users\emilh\Desktop\Data_split\Full data\AEPrec_sABRe_data_TPsABR21_BILATERAL_2021-Aug-18_13.09.55.xlsx'

try:
    excel_app=xw.App(visible=False)
    wb=excel_app.books.open(xl)
    for sheet in wb.sheets:
        sheet.api.Copy()
        wb_new=xw.books.active
        wb_new.save(f'{xl+sheet.name}.xlsx')
        wb_new.close()
finally:
    excel_app.quit()

In [ ]:
cwd=os.chdir(r'C:\\Users\\emilh\Desktop\\New_Data_split')
files=os.listdir(cwd)

df=pd.DataFrame()
for file in files:
    if file.endswith('.xlsx'):
        df=df.append(pd.read_excel(file), ignore_index=True)

df.head()
df.to_excel('data.xlsx')

In [14]:
file = pd.read_excel(r'C:\Users\emhm\Desktop\Simp_Data\data.xlsx')
file.insert(0, 'Signal', '1')

### Data without signal

In [12]:
no_sig=pd.read_excel(r'C:\Users\emhm\Desktop\No_sig_data\ABR_Noise.xlsx')
no_sig.insert(0, 'signal', '0')
no_sig.insert(0, 'ipsi_wa (nV)', 'ipsi_wa (nV)')
no_sig.to_excel('ABR_Noise_Edit.xlsx')

### Final data

In [47]:
cwd=os.chdir(r'C:\Users\emhm\Desktop\Final_Data')
files=os.listdir(cwd)

df=pd.DataFrame()
for file in files:
    if file.endswith('.xlsx'):
        df=df.append(pd.read_excel(file), ignore_index=True)

df.head()
df.to_excel('final_data.xlsx')

C:\Users\emhm\AppData\Local\Temp\ipykernel_24752\3059609829.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(pd.read_excel(file), ignore_index=True)
C:\Users\emhm\AppData\Local\Temp\ipykernel_24752\3059609829.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(pd.read_excel(file), ignore_index=True)


## Feature extraction

In [4]:
data=pd.read_excel(r'C:\Users\emhm\Desktop\Final_Data\final_data.xlsx')
X=data.drop('Label', axis=1)
y=data.loc[:, 'Label']

x_train, x_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=42)

print(X)

               1  Unnamed: 1  Unnamed: 2  Unnamed: 3  Unnamed: 4  Unnamed: 5  \
0       5.603800  -38.927500  -11.341600  -17.124500  -48.048800  -50.396700   
1     -43.212600  -79.867200   33.565600   15.791900  -72.675900    8.417500   
2     -46.965600   18.698800   98.799000   56.311100   30.194000  -20.405100   
3     -99.623400  115.402500   19.279300  -85.524100   23.319400   21.758900   
4    -129.249400   27.313500  -29.172700  -98.742900  -27.481700  -78.638100   
...          ...         ...         ...         ...         ...         ...   
1915   36.137741   13.023998   11.515466    1.477971  -18.800922  -67.927864   
1916   37.027374   13.442250   12.359756    0.885935  -20.103682  -66.844124   
1917   38.322655   13.239399   13.453218    0.431290  -22.010288  -65.040337   
1918   39.658405   12.517688   14.589395    0.308821  -24.157131  -62.847839   
1919         NaN   40.639149   11.476324   15.424708    0.668191  -26.218306   

      Unnamed: 6  Unnamed: 7  Unnamed: 

In [6]:
cfg = tsfel.get_features_by_domain()

feature_train = tsfel.time_series_features_extractor(cfg, x_train, fs=150)
feature_test = tsfel.time_series_features_extractor(cfg, x_test, fs=150)

*** Feature extraction started ***
60


IndexError: index 0 is out of bounds for axis 0 with size 0

In [35]:
feature_test

,0_Absolute energy,0_Area under the curve,0_Autocorrelation,0_Centroid,0_ECDF Percentile Count_0,0_ECDF Percentile Count_1,0_ECDF Percentile_0,0_ECDF Percentile_1,0_ECDF_0,0_ECDF_1,...,9_Wavelet variance_0,9_Wavelet variance_1,9_Wavelet variance_2,9_Wavelet variance_3,9_Wavelet variance_4,9_Wavelet variance_5,9_Wavelet variance_6,9_Wavelet variance_7,9_Wavelet variance_8,9_Zero crossing rate
0,1.137388e+06,75.288418,1.137388e+06,1.303597,76.0,307.0,-55.175,47.2437,0.002604,0.005208,...,3887.412137,4741.00782,4306.83268,4162.720583,4193.874468,4370.068473,4573.968522,4761.893343,4849.13791,174.0


## Feature selection

In [24]:
X, y = make_classification(
    n_features=20,
    n_informative=3,
    n_redundant=0,
    n_classes=2,
    n_clusters_per_class=2,
    random_state=42,
)

### Chi-Square

## Accuracy functions

In [ ]:
def get_confusion_matrix(y, y_pred, model):
    
    conf_matrix = confusion_matrix(y, y_pred)
    df_cm = pd.DataFrame(conf_matrix,index = ['Signal','No Signal'], columns = ['Signal','No Signal'])
    sns.set(font_scale = 1.4)
    sns.heatmap(df_cm, annot = True, cmap = 'OrRd', fmt = 'g', annot_kws={"size": 16}) # font size
    plt.ylabel('True labels')
    plt.xlabel('Predicted labels')
    plt.title('%s - Confusion Matrix' %model)
    #plt.savefig('confusion_matrix.png', dpi = 100, bbox_inches='tight' )
    plt.show()
    plt.close()

    return conf_matrix


def get_normalized_confusion_matrix(y, y_pred, model):
    
    conf_matrix = confusion_matrix(y, y_pred)
    conf_matrix = conf_matrix.astype(float)/conf_matrix.sum(axis = 1)[:,np.newaxis]
    df_cm = pd.DataFrame(conf_matrix,index = ['Signal','No Signal'], columns = ['Signal','No Signal'])
    sns.set(font_scale = 1.4)
    sns.heatmap(df_cm, annot = True, cmap = 'OrRd', fmt = '.2f',annot_kws={"size": 16}) # font size
    plt.ylabel('True labels')
    plt.xlabel('Predicted labels')
    plt.title('%s - Normalized Confusion Matrix' %model)
    #plt.savefig('confusion_matrix.png', dpi = 100, bbox_inches='tight' )
    plt.show()
    plt.close()
    
    return conf_matrix


# Based on Sensitivity and Recall 
# https://bmcgenomics.biomedcentral.com/articles/10.1186/s12864-019-6413-7
def get_matthews_corrcoef(y_pred, y_true):
    cov = np.cov(y_true, y_pred)
    if cov[0,0]==0. or cov[1,1]==0.:
        return 0.
    else:
        return cov[0,1]/np.sqrt(cov[0,0]*cov[1,1]) 
    
    
# Precision-Recall curve to evaluate a model
# PR curves are specifically tailored for the detection of rare events
def get_precision_recall_curve(y_true, y_pred):
    #precision, recall, thresholds = precision_recall_curve(y_test, probs)
    return plot_precision_recall_curve(classifier, x_test, y_test)
  
    
    
# ROC curves can sometimes be misleading in some very imbalanced applications
def get_roc_curve(y_true, y_pred_proba, model):
    
    #keep probabilities for the positive outcome only
    y_pred_proba = y_pred_proba[:,1]
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
    
    auc = roc_auc_score(y_true, y_pred_proba)
    print('ROC AUC=%.3f' % (auc))
    
    plt.plot([0,1],[0,1],linestyle = '--',lw = 0.8,color = 'black', label='Luck')
    plt.plot(fpr, tpr, marker='.', lw=2, color='red', label=r'ROC (AUC = %0.3f )' % (auc))
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(r'%s - ROC curve' % (model))
    plt.legend(loc = 'lower right')
    plt.show()
    
    return auc
    

## Random forrest

In [2]:
rand_forrest_class=RandomForestClassifier(n_estimators=400,
                                            max_leaf_nodes=16,
                                            n_jobs=-1)

rand_forrest_class.fit(x_train,y_train)

NameError: name 'x_train' is not defined

In [ ]:
y_pred=rand_forrest_class.predict(x_test)

In [30]:
accuracy_score(y_test, y_pred)

1.0

In [ ]:
rand_forrest_class=RandomForestClassifier(n_estimators=400,
                                            max_leaf_nodes=16,
                                            n_jobs=-1)
rand_forrest_class.fit(x_train, y_train)

#number of trees used
print('Number of trees used: ', rand_forrest_class.n_estimators)

# Predicting the target on training dataset
predict_train=rand_forrest_class.predict(feature_train)
print('Target on train data', predict_train)

#Accuracy score
acurracy_train=accuracy_score(y_train, predict_train)
print('Accuracy_score on training set: ', acurracy_train)

# Predict the target on test set
predict_test=rand_forrest_class.predict(feature_test)
print('Target on test data', predict_test)

#Accuracy score on test data
acurracy_test=accuracy_score(y_test, predict_test)
print('Accuracy_score on test data', acurracy_test)

### Adaboost

In [ ]:
ada_clf=AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
                            n_estimators=100,
                            algorithm='SAMME',
                            learning_rate=1.0)

ada_clf.fit(x_train, y_train)

ada_clf.estimator_weights_

In [27]:
y_pred=ada_clf.predict(x_test)

accuracy_score(y_test, y_pred)

1.0